In [ ]:
### Installation

In [1]:
import sys; modules = list(sys.modules.keys())
for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None

%pip install "unsloth==2025.2.4" vllm
%pip install -q --upgrade pillow
%pip install -q transformers==4.31.0
%pip install -q rouge_score bert_score datasets evaluate scikit-learn sentence_transformers sacremoses
# If you are running this notebook on local, you need to install `diffusers` too
%pip install -q diffusers
# Temporarily install a specific TRL nightly version
%pip install -q git+https://github.com/huggingface/trl.git@e95f9fb74a3c3647b86f251b7e230ec51c64b72b
%pip install --upgrade --force-reinstall --no-cache-dir --no-deps unsloth unsloth_zoo    

  Using cached unsloth-2025.2.4-py3-none-any.whl.metadata (57 kB)
Using cached unsloth-2025.2.4-py3-none-any.whl (181 kB)
  Attempting uninstall: unsloth
    Found existing installation: unsloth 2025.3.10
    Uninstalling unsloth-2025.3.10:
      Successfully uninstalled unsloth-2025.3.10
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.31.0 which is incompatible.
trl 0.15.0.dev0 requires transformers>=4.46.0, but you have transformers 4.31.0 which is incompatible.
unsloth 2025.2.4 requires transformers!=4.47.0,>=4.46.1, but you have transformers 4.31.0 which is incompatible.
unsloth-zoo 2025.3.9 requires transformers!=4.47.0,>=4.46.1, but 

In [2]:
!pip install diffusers laio

In [ ]:
# Use `PatchFastRL` before all functions to patch GRPO and other RL algorithms

In [4]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/cdsw/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 03-19 15:07:56 __init__.py:190] Automatically detected platform cuda.


2025-03-19 15:08:00,175	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [ ]:
#Load up `Llama 3.1 8B Instruct`, and set parameters

In [5]:
from unsloth import is_bfloat16_supported
import torch
max_seq_length = 1024 # Can increase for longer reasoning traces
lora_rank = 64 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "meta-llama/meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.5, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

==((====))==  Unsloth 2025.3.16: Fast Llama patching. Transformers: 4.49.0. vLLM: 0.7.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/meta-llama-3.1-8b-instruct-unsloth-bnb-4bit with actual GPU utilization = 49.53%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 1024. Num Sequences = 128.
Unsloth: vLLM's KV Cache can use up to 0.97 GB. Also swap space = 5 GB.
WARNING 03-19 15:09:17 config.py:2386] Casting torch.bfloat16 to torch.float16.
INFO 03-19 15:09:37 config.py:542] This model supports multiple tasks: {'classify', 'generate', 'score',

[W319 15:09:41.849108997 CUDAAllocatorConfig.h:28] Warning: expandable_segments not supported on this platform (function operator())


INFO 03-19 15:09:41 loader.py:1102] Loading weights with BitsAndBytes quantization.  May take a while ...
INFO 03-19 15:09:42 weight_utils.py:252] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:14<00:00, 14.58s/it]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:14<00:00, 14.58s/it]

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:02<00:00,  2.07s/it]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:02<00:00,  2.07s/it]



INFO 03-19 15:09:59 model_runner.py:1115] Loading model weights took 5.5976 GB
INFO 03-19 15:09:59 punica_selector.py:18] Using PunicaWrapperGPU.
INFO 03-19 15:10:04 worker.py:267] Memory profiling takes 4.43 seconds
INFO 03-19 15:10:04 worker.py:267] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.50) = 7.30GiB
INFO 03-19 15:10:04 worker.py:267] model weights take 5.60GiB; non_torch_memory takes 0.03GiB; PyTorch activation peak memory takes 0.60GiB; the rest of the memory reserved for KV Cache is 1.08GiB.
INFO 03-19 15:10:04 executor_base.py:110] # CUDA blocks: 551, # CPU blocks: 2560
INFO 03-19 15:10:04 executor_base.py:115] Maximum concurrency for 1024 tokens per request: 8.61x
INFO 03-19 15:10:08 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occur

Capturing CUDA graph shapes: 100%|██████████████| 19/19 [00:22<00:00,  1.16s/it]

INFO 03-19 15:10:30 model_runner.py:1562] Graph capturing finished in 22 secs, took 0.52 GiB
INFO 03-19 15:10:30 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 31.35 seconds



Unsloth 2025.3.16 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [6]:
!nvidia-smi

Wed Mar 19 15:11:06 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.03             Driver Version: 550.144.03     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       On  |   00000000:00:1E.0 Off |                    0 |
| N/A   33C    P0             32W /   70W |    8577MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
#Data Prep for supporting use-case-specific training data and setting up reward functions. Here, we define rewards for semantic correctness, perplexity and tag presence

#For data prep and all reward functions, we leverage an open-source dataset - FreedomIntelligence/medical-o1-reasoning-SFT, a dataset used to fine-tune HuatuoGPT-o1, a medical LLM designed for advanced medical reasoning

In [7]:
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split

SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

def get_medical_questions(split="train") -> Dataset:
    # Load the raw dataset from the hub
    data = load_dataset('FreedomIntelligence/medical-o1-reasoning-SFT', 'en')[split]
    # Convert to a pandas DataFrame for easier splitting
    df = data.to_pandas()

    if split == "train":
        # Split the data into 70% train and 30% temporary data
        train_df, temp_df = train_test_split(df, test_size=0.01, random_state=42)
        # Split the temporary data equally into eval and test (each 2.5% of the total) 
        eval_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

        # Convert back to Hugging Face Datasets
        train_dataset = Dataset.from_pandas(train_df)
        eval_dataset = Dataset.from_pandas(eval_df)
        test_dataset = Dataset.from_pandas(test_df)

        # Define a mapping function to create the prompt and answer fields
        def map_fn(x):
            return {
                'prompt': [
                    {'role': 'system', 'content': SYSTEM_PROMPT},
                    {'role': 'user', 'content': x['Question']}
                ],
                'answer': x['Response'],  # The reference answer
                'question': x['Question']
            }

        # Apply the mapping function to each split and remove unnecessary columns
        train_dataset = train_dataset.map(map_fn).remove_columns(['Question', 'Complex_CoT', 'Response', '__index_level_0__'])
        eval_dataset = eval_dataset.map(map_fn).remove_columns(['Question', 'Complex_CoT', 'Response', '__index_level_0__'])
        test_dataset = test_dataset.map(map_fn).remove_columns(['Question', 'Complex_CoT', 'Response', '__index_level_0__'])

        return train_dataset, eval_dataset, test_dataset  # Return all three splits
    else:
        # For non-"train" splits, just map over the data and remove unnecessary columns
        data = data.map(lambda x: {
            'prompt': [
                {'role': 'system', 'content': SYSTEM_PROMPT},
                {'role': 'user', 'content': x['Question']}
            ],
            'answer': x['Response'],
            'question': x['Question']
        }).remove_columns(['Question', 'Complex_CoT', 'Response', '__index_level_0__'])
        return data

# To get the three splits, call:
train_dataset, eval_dataset, test_dataset = get_medical_questions(split="train")
# train_dataset = train_dataset.select(range(3)), 
train_dataset, eval_dataset, test_dataset

Map: 100%|███████████████████████████| 127/127 [00:00<00:00, 8677.07 examples/s]


(Dataset({
     features: ['prompt', 'answer', 'question'],
     num_rows: 25117
 }),
 Dataset({
     features: ['prompt', 'answer', 'question'],
     num_rows: 127
 }),
 Dataset({
     features: ['prompt', 'answer', 'question'],
     num_rows: 127
 }))

In [ ]:
# Reward Functions

#The semantic correctness reward uses a cross-encoder model (cross-encoder/stsb-roberta-base) to evaluate the similarity between generated responses and reference answers. When no valid answer is extracted (i.e., an empty response), the function assigns a reward of -1.0 to indicate failure in producing an answer.

#The perplexity calculation is handled by the PerplexityCalculator class which uses BioGPT (microsoft/biogpt) to measure the fluency and linguistic quality of responses. Note that BioGPT typically demands extra GPU resources due to its computational intensity. If a response is empty or an error occurs (e.g., a NaN loss), the function returns a default perplexity value of -1.0 to signal the issue.

#The combined reward function aggregates the rewards from semantic correctness and perplexity. The rewards are weighted, with semantic similarity and perplexity contributing most significantly to form a final score. The final reward is clamped between -1 and 1, ensuring that any instance where no valid answer is found (or other issues arise) results in a -1 reward.

In [8]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import CrossEncoder
from typing import List
import re

# ------ Device Configuration ------
main_device = "cuda" if torch.cuda.is_available() else "cpu"
reward_device = "cuda" if torch.cuda.is_available() else "cpu"

# ------ Semantic Correctness Reward ------
def semantic_correctness(responses: List[str], answers: List[str]) -> List[float]:
    """Calculate semantic similarity using cross-encoder"""
    model = CrossEncoder('cross-encoder/stsb-roberta-base', device=reward_device)
    with torch.no_grad():            
        inputs = list(zip(responses, answers))
        similarities = model.predict(inputs, show_progress_bar=False).tolist()
        # Set similarity to -1 if the response is an empty string
        similarities = [-1.0 if response == "" else similarity for response, similarity in zip(responses, similarities)]
        return similarities

# ------ Efficient Perplexity Calculation ------
class PerplexityCalculator:
    def __init__(self, model_name="microsoft/biogpt", device=reward_device):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.device = device
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.model = AutoModelForCausalLM.from_pretrained(model_name).to(self.device)
        self.model.eval()

    def calculate(self, texts: List[str], batch_size=8) -> List[float]:
        perplexities = []
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i+batch_size]
            try:
                if not batch: continue
                
                encodings = self.tokenizer(
                    batch, 
                    return_tensors="pt", 
                    padding=True, 
                    truncation=True,
                    max_length=200
                ).to(self.device)
                
                with torch.no_grad():
                    outputs = self.model(**encodings, labels=encodings.input_ids)
                
                loss = outputs.loss
                if torch.isnan(loss):
                    raise ValueError("NaN loss encountered")
                
                batch_perplexity = torch.exp(loss).repeat(len(batch)).cpu().tolist()
                # Set perplexity to -1 if the input is an empty string
                batch_perplexity = [-1.0 if text == "" else perplex for text, perplex in zip(batch, batch_perplexity)]
                perplexities.extend(batch_perplexity)
                                
            except Exception as e:
                print(f"Error in batch {i//batch_size}: {str(e)}")
                perplexities.extend([1000.0] * len(batch))
        
        return perplexities

perplexity_calculator = PerplexityCalculator()

# ------ Tag Presence Reward ------
def tag_presence_reward(completions: List[dict]) -> List[float]:
    """Reward for presence of <reasoning> and <answer> tags"""
    rewards = []
    for completion in completions:
        content = completion[0]['content']
        has_reasoning = bool(re.search(r'<reasoning>.*?</reasoning>', content, re.DOTALL))
        has_answer = bool(re.search(r'<answer>.*?</answer>', content, re.DOTALL))
        reward = 0.5 * has_reasoning + 0.5 * has_answer
        rewards.append(reward)
    return rewards

# ------ Combined Reward Function ------
def combined_reward_func(
    prompts, completions, answer, **kwargs
) -> List[float]:
    # Extract generated responses
    responses = []
    valid_indices = []
    for idx, completion in enumerate(completions):
        try:
            generated_content = completion[0]['content'].strip()

            # Extract only the <answer> content using regex
            answer_match = re.search(r'<answer>(.*?)</answer>', generated_content, re.DOTALL)
            if answer_match:
                generated_content = answer_match.group(1).strip()
            else:
                responses.append("")  # Append empty string to maintain index consistency
                valid_indices.append(idx)
                continue
            
            # Skip if empty or just repeating the prompt
            user_prompt = prompts[idx][-1]['content']
            if not generated_content or generated_content == user_prompt:
                # print(f"generated_content: continue")
                responses.append("")  # Append empty string to maintain index consistency
                valid_indices.append(idx)
                continue
                
            responses.append(generated_content)
            # print(f"generated_content3: {generated_content}")
            valid_indices.append(idx)
        except (KeyError, IndexError):
            responses.append("")  # Append empty string to maintain index consistency
            valid_indices.append(idx)
            continue
    
    if not responses:
        return [-1.0] * len(completions)
    
    # Calculate rewards
    try:
        # Use raw answers without tag processing
        processed_answers = answer
        # print(f"processed_answers: {processed_answers[0]}")
        
        similarities = semantic_correctness(responses, [processed_answers[i] for i in valid_indices])
        perplexities = perplexity_calculator.calculate([processed_answers[i] for i in valid_indices])
        tag_rewards = tag_presence_reward([completions[i] for i in valid_indices])
    except Exception as e:
        print(f"Reward calculation error: {str(e)}")
        return [-1.0] * len(completions)
    
    # Convert to tensors with stability
    sim_scores = torch.nan_to_num(torch.tensor(similarities), nan=0.0)
    perplex_scores = torch.nan_to_num(torch.tensor(perplexities), nan=1000.0)
    tag_scores = torch.tensor(tag_rewards)
        
    # Perplexity reward calculation
    perplex_rewards = 1 / (perplex_scores / (perplex_scores.mean() + 1e-9))
    
    # Normalize with stability
    score_range = perplex_rewards.max() - perplex_rewards.min()
    if score_range < 1e-6:
        perplex_rewards_normalized = torch.ones_like(perplex_rewards) * 0.5
    else:
        perplex_rewards_normalized = (perplex_rewards - perplex_rewards.min()) / score_range
    
    # Combine scores with validation
    combined = [
        0.5 * sim.item() + 0.4 * pr.item() + 0.1 * tag.item()
        for sim, pr, tag in zip(sim_scores, perplex_rewards_normalized, tag_scores)
        if not torch.isnan(sim) and not torch.isnan(pr) and not torch.isnan(tag)
    ]
    
    # Map back to original indices
    final_rewards = [-1.0] * len(completions)
    for idx, reward in zip(valid_indices, combined):
        final_rewards[idx] = max(min(reward, 1.0), -1.0)  # Clamp between -1 and 1
    
    assert len(final_rewards) == len(completions), "Reward mapping error"
    return final_rewards

In [ ]:
#Train the model
#Now set up GRPO Trainer and all configurations

In [13]:
per_device_train_batch_size = 1
gradient_accumulation_steps = 2 # 2 for quicker iterations, 4 otherwise

total_steps = 200  # Changed from max steps of 1000
num_checkpoints = 4

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    use_vllm=True,  # use vLLM for fast inference!
    learning_rate=5e-6,
    adam_beta1=0.9,
    adam_beta2=0.99,
    weight_decay=0.1,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    optim="adamw_8bit",
    logging_steps=1,
    bf16=is_bfloat16_supported(),
    fp16=not is_bfloat16_supported(),
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    num_generations=5,  # Decrease if out of memory
    max_prompt_length=128,  # Updated by lowering to 128 from 512 to balance longer input prompts with training time requirements
    max_completion_length=128,
    max_steps=total_steps,
    save_steps=int(total_steps // num_checkpoints),
    max_grad_norm=0.1,
    report_to="none",  # Can use Weights & Biases
    output_dir="grpo_outputs",
    save_strategy="steps",
)

In [ ]:
#Let's run the trainer. If you scroll, you'll see a table of rewards. The goal is to see the reward column increase.
#You might have to wait 150 to 200 steps for any action. 

In [14]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        combined_reward_func
    ],
    args = training_args,
    train_dataset = train_dataset,
)

In [ ]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 25,117 | Num Epochs = 1 | Total steps = 200
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 2 x 1) = 2
 "-____-"     Trainable parameters = 167,772,160/8,000,000,000 (2.10% trained)


Step,Training Loss,reward,reward_std,completion_length,kl,rewards / combined_reward_func
1,0.003100,0.369562,0.261547,120.000000,0.078429,0.369562
2,0.003900,0.181849,0.024764,121.500000,0.097199,0.181849
3,0.003200,0.614691,0.031964,108.100002,0.078781,0.614691
4,0.005600,0.670583,0.013329,96.600002,0.139568,0.670583
5,0.004200,0.666211,0.018930,91.000002,0.104646,0.666211
6,0.003500,0.509316,0.029912,82.500000,0.086823,0.509316
7,0.004900,0.567360,0.023304,111.000000,0.121934,0.567360
8,0.003800,0.556397,0.024406,114.900002,0.095382,0.556397
9,0.003100,0.403424,0.224104,122.299999,0.078109,0.403424
10,0.003800,0.211518,0.236788,90.500000,0.094974,0.211518


In [ ]:
#Inference
#Now let's try the model we just trained: First, let's first try the model without any GRPO trained

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "Is Aspirin good for cardio vascular function?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

In [ ]:
output

In [ ]:
#And now with the LoRA we just trained with GRPO - we save the LoRA first:

In [ ]:
#Now we load the LoRA and test:

In [ ]:
model.save_lora("grpo_saved_lora")

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "Is Aspirin good for cardio vascular function?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

In [ ]:
output

In [ ]:
model.save_pretrained_merged("model", tokenizer)

In [ ]:
# To test the model quickly, skip fine-tuning and use the pre-trained checkpoint. Pre-trained checkpoints are only included for demo purposes.
# Load pre-trained Unsloth checkpoint and generate output

In [ ]:
from unsloth import FastLanguageModel
from vllm import SamplingParams

# Load pretrained model only, skip PEFT/fine-tune setup
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/meta-llama-3.1-8b-instruct-bnb-4bit",
    max_seq_length=256,
    load_in_4bit=True,
    fast_inference=True,
    gpu_memory_utilization=0.8,
)

# Prepare prompt directly
prompt = tokenizer.apply_chat_template([
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Explain why the sky is blue."},
], tokenize=False, add_generation_prompt=True)

# Set sampling/generation params
sampling_params = SamplingParams(
    temperature=0.8,
    top_p=0.95,
    max_tokens=1024,
)

# Generate (No training/fine-tuning needed)
response = model.fast_generate(prompt, sampling_params=sampling_params)[0].outputs[0].text
print("Model Output:", response)
